In [12]:
import yake
import sys
from bs4 import BeautifulSoup
from bs4.element import Comment
import urllib.request
import requests
import time
from IPy import IP
from collections import defaultdict
import itertools  #used to slice a dictionary

import json
import pickle

import pprint

In [13]:
#Functions used for extraction of text from the url provided

def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True

def text_from_html(body):
    soup = BeautifulSoup(body, 'html.parser')
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)  
    return u" ".join(t.strip() for t in visible_texts)

def text_from_title(body):
    soup = BeautifulSoup(body, 'html.parser')
    title_text = soup.find('title').text
    return title_text

In [14]:
def keywordExtractor(url):
    url = url.strip()
    headers={'User-Agent': 'Mozilla/5.0'}
    resp = requests.get(url, headers=headers, timeout=3) 
    if resp.status_code >= 300:
        return
    html = resp.text
    text = text_from_html(html)
    title_text = text_from_title(html).lower()
    max_ngram_size = 1
    simple_kwextractor = yake.KeywordExtractor(lan="en", n = max_ngram_size, dedupLim=0.9, dedupFunc='seqm', windowsSize=1, top=20, features=None)
    keywords = simple_kwextractor.extract_keywords(text)
    title_keywords = simple_kwextractor.extract_keywords(title_text)
    title_words = dict([(t[1], t[0]) for t in title_keywords])  #to swap the term and its score to get {term:score}
    title_words = title_words.keys()
    
    keywords2 = []
    for kw in keywords:
        #since the order of term and score is reversed
        if kw[1] in title_words:
            kw2 = (kw[0]/4, kw[1])
        else:
            kw2 = kw
        keywords2.append(kw2)   #should be inside for loop so that all words are appended
    keywords2 = dict([(t[1], t[0]) for t in keywords2]) #to make a dictionary(as given in description) and also to reverse the order to {term:score}
    return keywords2

In [15]:
# load new history created since last update

path_history = './data/history.data'

try:
	with open(path_history, 'rb') as f:
		history_dict = pickle.load(f)
except:
	history_dict = {}
    
pprint.pprint(history_dict)
    
# looks like: 
# {
#     'http://localhost:8888/lab': {'scroll_dist': 0, 'time_spent': 27},
    
#     'https://codeforces.com/': {'scroll_dist': 1187.199951171875, 'time_spent': 6},
    
#     'https://github.com/kanishk509/Multi-dimensional-graph-based-search-customization': {'scroll_dist': 0,
#                                                                                       'time_spent': 8},
    
#     'https://stackoverflow.com/questions/2260446/how-to-iterate-through-dictionary-and-change-values': {'scroll_dist': 0,
#                                                                                                      'time_spent': 3},
    
#     'https://web.whatsapp.com/': {'scroll_dist': 0, 'time_spent': 4}
# }

{'http://localhost:8888/lab': {'scroll_dist': 0, 'time_spent': 27},
 'https://codeforces.com/': {'scroll_dist': 1187.199951171875, 'time_spent': 6},
 'https://github.com/kanishk509/Multi-dimensional-graph-based-search-customization': {'scroll_dist': 0,
                                                                                      'time_spent': 8},
 'https://stackoverflow.com/questions/2260446/how-to-iterate-through-dictionary-and-change-values': {'scroll_dist': 0,
                                                                                                     'time_spent': 3},
 'https://web.whatsapp.com/': {'scroll_dist': 0, 'time_spent': 4}}


In [25]:
# keywords database structure

keywords_db = {
    'python' : {
        'title'    : 1.0, 
        'body'     : 1.0, 
        'bookmark' : 1.0, 
        'urls'     : ['https://docs.python.org/3/tutorial/index.html', ]
    },
}

#load keywords database from json file

filename = "./data/keywords_db.data"

# try:
#     with open(filename, 'rb') as f:
#         keywords_db = pickle.load(f)
# except:
#     keywords_db = {}


In [17]:
# new keywords that we need to add as nodes/add edges for 
# in our clustering graph.

new_keywords_to_cluster = [['a', 'b'], ['c']]

In [26]:
def process_new_history(history_dict):
    
    for url in history_dict:
        url_time = history_dict[url]['time_spent']
        url_scroll = history_dict[url]['scroll_dist']
        if url_scroll == 0:
            # some default value
            url_scroll = 100.0

        try:
            # taking 5 most significant keywords
            keywords_list = keywordExtractor(url).items()
            keywords_list = sorted(keywords_list, key=lambda key: key[1])
            keywords_list = keywords_list[:5]
            
            # add keywords to be clustered
            new_keywords_to_cluster.append(
                [word for (word, value) in keywords_list])
            
            # add code to make new entry for keyword in keyword_db 
            # or if already present: 
            #    1) update score
            #    2) add url to the keyword's list of urls
            
            #for kw in keywords[:5]:
                # code to update score of keyword
                
        except:
            continue
    

In [23]:
process_new_history(history_dict)

In [ ]:
# save keywords to cluster
with open('./data/to_cluster.data', 'wb') as outfile:
    pickle.dump(new_keywords_to_cluster, outfile)

In [ ]:
# save keywords database
with open('./data/keyword_db.data', 'wb') as outfile:
    pickle.dump(keywords_db, outfile)